In [1]:
# Global settings
import time
import uuid

from datasets.data_loader import load_ppb

from slim.algorithms.GP.gp import GP
from slim.algorithms.GP.operators.mutators import mutate_tree_subtree
from slim.algorithms.GP.representations.tree_utils import tree_depth, tree_pruning
from slim.config.gp_config import *
from slim.utils.logger import log_settings
from slim.utils.utils import get_terminals, validate_inputs, train_test_split


In [2]:
X, y = load_ppb(X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5)
print(X_val.shape, X_test.shape, y_val.shape, y_test.shape)


torch.Size([26, 626]) torch.Size([26, 626]) torch.Size([26]) torch.Size([26])


In [3]:
# Problem instance
TERMINALS = get_terminals(X_train)

pop_size = 500

init_depth = 3
gp_pi_init["TERMINALS"] = TERMINALS
gp_pi_init["init_pop_size"] = pop_size
gp_pi_init["init_depth"] = init_depth


In [4]:
# GP
p_xo = 0.8
gp_parameters["p_xo"] = p_xo
gp_parameters["p_m"] = 1 - gp_parameters["p_xo"]
gp_parameters["pop_size"] = pop_size
gp_parameters["mutator"] = mutate_tree_subtree(
    gp_pi_init['init_depth'], TERMINALS, CONSTANTS, FUNCTIONS, p_c=gp_pi_init['p_c']
)


In [5]:
# Log
log_path = os.path.join(os.getcwd(), "log", "gp.csv")


In [6]:
# Solve settings
elitism = True
n_elites = 1
algo = 'StandardGP'
dataset_name = 'PPB'
unique_run_id = uuid.uuid1()
max_depth = 17
n_iter = 1000
gp_solve_parameters['run_info'] = [algo, unique_run_id, dataset_name]
gp_solve_parameters["log_path"] = log_path
gp_solve_parameters["elitism"] = elitism
gp_solve_parameters["n_elites"] = n_elites
gp_solve_parameters["max_depth"] = max_depth
gp_solve_parameters["n_iter"] = n_iter
gp_solve_parameters["tree_pruner"] = tree_pruning(
    TERMINALS=TERMINALS, CONSTANTS=CONSTANTS, FUNCTIONS=FUNCTIONS, p_c=gp_pi_init["p_c"]
)
gp_solve_parameters['depth_calculator'] = tree_depth(FUNCTIONS=FUNCTIONS)
gp_solve_parameters["test_elite"] = True


In [7]:
# GP instance
seed = 1
optimizer = GP(pi_init=gp_pi_init, **gp_parameters, seed=seed)


In [8]:
# Solve
n_iter = 250
gp_solve_parameters["n_iter"] = n_iter
optimizer.solve(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    curr_dataset=dataset_name,
    **gp_solve_parameters
)


Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     PPB                 |       0      |   34.88151168823242   |   35.582557678222656     |   0.17404794692993164  |      3           |
|     PPB                 |       1      |   30.97121238708496   |   29.513381958007812     |   0.06665396690368652  |      3           |
|     PPB                 |       2      |   30.97121238708496   |   29.513381958007812     |   0.06482887268066406  |      3           |
|     PPB                 |       3      |   30.86421012878418   |   30.11235809326172      |   0.061927080154418945 |      3           |
|     PPB        

In [9]:
log_settings(
    path=log_path[:-4] + "_settings.csv",
    settings_dict=[gp_solve_parameters,
                   gp_parameters,
                   gp_pi_init,
                   settings_dict],
    unique_run_id=unique_run_id,
)


In [10]:
final_tree = optimizer.elite


In [11]:
predictions = final_tree.predict(X_test)
print('RMSE test: {}\n'.format(float(rmse(y_true=y_test, y_pred=predictions))))

final_tree.print_tree_representation()


RMSE test: 31.302658081054688

add(
  add(
    x492
    divide(
      x548
      divide(
        x270
        add(
          x207
          multiply(
            x318
            add(
              x526
              divide(
                x91
                subtract(
                  x592
                  multiply(
                    x127
                    divide(
                      x525
                      multiply(
                        x410
                        add(
                          x479
                          subtract(
                            x592
                            divide(
                              x308
                              x285
                            )
                          )
                        )
                      )
                    )
                  )
                )
              )
            )
          )
        )
      )
    )
  )
  add(
    x492
    add(
      x526
      add(
        x15
     

In [12]:
final_tree.node_count

61